## 행정안전부 도로명주소 API 활용

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key 가져오기

In [2]:
with open('keys/도로명주소apiKey.txt') as file:
    road_key = file.read()

- URL 만들기

In [3]:
# requests get method를 이용하여 데이터를 보내는 경우에는 한글을 반드시 인코딩해주어야 함
quote('영등포역')

'%EC%98%81%EB%93%B1%ED%8F%AC%EC%97%AD'

In [8]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("영등포역")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

- Open API를 호출하여 결과 받기

In [9]:
result = requests.get(url)
result.status_code

200

In [10]:
result.text[:200]

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"8","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"B843 Gyeongin-ro, Yeongdeungpo-gu, Seoul","rn":"경'

- JSON data를 python에서 읽을 수 있도록 변환

In [11]:
res = json.loads(result.text)

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [13]:
res.keys()

dict_keys(['results'])

In [14]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [15]:
type(res['results']['juso'])

list

In [16]:
res['results']['juso'][0].keys()

dict_keys(['detBdNmList', 'engAddr', 'rn', 'emdNm', 'zipNo', 'roadAddrPart2', 'emdNo', 'sggNm', 'jibunAddr', 'siNm', 'roadAddrPart1', 'bdNm', 'admCd', 'udrtYn', 'lnbrMnnm', 'roadAddr', 'lnbrSlno', 'buldMnnm', 'bdKdcd', 'liNm', 'rnMgtSn', 'mtYn', 'bdMgtSn', 'buldSlno'])

In [17]:
# 원하는 결과: 도로명 주소
res['results']['juso'][0]['roadAddr']

'서울특별시 영등포구 경인로 지하843 (영등포동3가)'

- 영등포역, 신도림역, 당산역, 여의도역, 영등포구청역

In [19]:
stations = '영등포역, 신도림역, 당산역, 여의도역, 영등포구청역'.split(', ')
stations

['영등포역', '신도림역', '당산역', '여의도역', '영등포구청역']

In [20]:
road_addr_list = []
for station in stations:
    params2 = f'keyword={quote(station)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text)
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [21]:
df = pd.DataFrame({
    '이름': stations, '주소': road_addr_list
})
df

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)


In [23]:
df.to_csv('data/영등포소재 지하철역.csv', index=False)
pd.read_csv('data/영등포소재 지하철역.csv')

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)
